# Хакатон_2023

Создаем DataFrame с необходимой информацией для:
* построения графа,
* получения необходимой статистики,
* получения полной разметки для дальнейшего обучения модели
* для поиска NER на новых docx 
* выявления ошибок в документахНовый пункт

### Ипортируем необходимые библиотеки

In [ ]:
! pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=68e8635d0a980efc453c7be04288d222e93a806efe8bc85104954d16e5831ab1
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [ ]:
import os
import zipfile

import docx
from termcolor import colored

import re

# для работы с графами
import networkx as nx

import json
import pandas as pd

In [ ]:
! pip install pullenti-wrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 18.1 MB/s eta 0:00:00


In [ ]:
from pullenti_wrapper.processor import (
    Processor,
    ORGANIZATION,
    DATE,
    GEO,
    PERSON,
    MONEY,
    PHONE, 
    ADDRESS
)

In [ ]:
!tar -xzf /content/drive/MyDrive/Хакатон_2023/full_dataset/docx.tar
!tar -xf /content/drive/MyDrive/Хакатон_2023/full_dataset/json.tar

In [ ]:
!python -m spacy download ru_core_news_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-17 17:15:48.500420: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 64.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=866b26a9cce19c804cdbc2308c0bcb610fb851cff955e25147da912e96ab80c0
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
✔ Download and installation 

In [ ]:
import spacy
nlp = spacy.load('ru_core_news_lg')

# Данные из json добавляем в pd.DataFrame

In [ ]:
all_tasks = []

path_normalized = os.path.normpath("/content/json")
for dirpath, dirnames, filenames in os.walk(path_normalized):
    for file in filenames:
        # json_names.append(int(file.rstrip('.json')))
        with open(os.path.normpath(path_normalized + "/" + file), 'r') as json_file:
              tasks = json.load(json_file)
              tasks["id"] = int(file.rstrip('.json'))
              all_tasks.append(tasks)   

In [ ]:
df = pd.json_normalize(all_tasks)


In [ ]:
df.set_index('id', inplace=True)
df.sort_index(inplace=True) # сортируем по индеку=id 

In [ ]:
df.columns = df.columns.str.replace("Tasks.", "") # укорачиваем названия столбцов

<ipython-input-112-8759b8cd685a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace("Tasks.", "") # укорачиваем названия столбцов


# Тексты из документов

In [ ]:
all_text = []
file_names = []

path_normalized = os.path.normpath("/content/docx")
for dirpath, dirnames, filenames in os.walk(path_normalized):
    for file in filenames:
        file_names.append(int(file.rstrip('.docx')))
        text_doc = []
        doc = docx.Document(os.path.normpath(path_normalized + "/" + file))
        # собираем документ в список по абзацам и удаляем пустые абзацы
        for par in doc.paragraphs:
            if par.text != "":
                text_doc.append([par.text.replace("\n", "")])
        all_text.append(text_doc)
        
raw_df = pd.DataFrame({"id": file_names, "text": all_text}, columns=["id", "text"])
raw_df["number_of_paragraphs"] = raw_df["text"].str.len()
raw_df.set_index('id', inplace=True)
raw_df.sort_index(inplace=True) # сортируем по индеку=id 
print(raw_df.tail(5))

                                                   text  number_of_paragraphs
id                                                                           
9995  [[ООО "Мегасофт"], [Приказ], [Для организации ...                     8
9996  [[ООО "Мегасофт"], [Приказ], [В связи с оконча...                    22
9997  [[ООО "Мегасофт"], [Приказ], [В связи с оконча...                    14
9998  [[ООО "Мегасофт"], [Приказ], [Для организации ...                    18
9999  [[ООО "Мегасофт"], [Приказ], [Для организации ...                     8


In [ ]:
# выделяем "сырые" сущности
col_one_list = [''.join(i[0]) for i in raw_df['text']]
df["Organization"] = col_one_list # название организации
first_column = df.pop("Organization")
df.insert(0, "Organization", first_column) # для удобства перемещаем на 0 место столбец 

title_list = [''.join(i[1]) for i in raw_df['text']]
df["Title"] = title_list # заголовок док-та
first_column = df.pop("Title")
df.insert(2, "Title", first_column) # для удобства перемещаем на 0 место столбец 

signatory_and_date = [''.join(i[-1]) for i in raw_df['text']]
raw_df["signatory_and_date"] = signatory_and_date # подписант док-та + дата подписания

# Добавляем в данные из json информацию из docx

# Подписант документа с должностью

In [ ]:
def get_signatory(x):
    for index, letter in enumerate(x):
      if letter.isdigit():
          signatory= x[:index]
          date = x[index:]
          break

    return signatory


In [ ]:
df["Signatory"] = raw_df["signatory_and_date"].apply(lambda x: get_signatory(x)) # добавляем подписанта
first_column = df.pop("Signatory")
df.insert(1, "Signatory", first_column) # для удобства перемещаем на 0 место столбец 

In [ ]:
raw_df.head(3)

,text,number_of_paragraphs,signatory_and_date
id,,,
0,"[[ООО ""Мегасофт""], [Приказ], [Для организации ...",6,и.о. генерального директора Сердитый А.И.11 ап...
1,"[[ООО ""Мегасофт""], [Приказ], [Для организации ...",10,и.о. генерального директора Сердитый А.И.03 ию...
2,"[[ООО ""Мегасофт""], [Приказ], [В связи с оконча...",16,генеральный директор Созонов С.В.08 окт 2014 г.


In [ ]:
df.head(3)

,Signatory,Organization,Task1.task_text,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,Task2.task_responsibles_people,Task2.task_responsibles_groups,...,Task8.task_text,Task8.task_responsibles_people,Task8.task_responsibles_groups,Task8.task_deadline,Global_supervisor,Global_deadline,Task9.task_text,Task9.task_responsibles_people,Task9.task_responsibles_groups,Task9.task_deadline
id,,,,,,,,,,,,,,,,,,,,,
0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"[19 апр 15, 1429383600.0]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[Ответственность за исполнение распоряжения во...,"[11 апреля 2013 года, 1365616800.0]",NaN,NaN,NaN,NaN
1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"[24 октября 12, 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,Контроль над выполнением приказа возложить на ...,Служба ИТ,...,NaN,NaN,NaN,NaN,[Ответственность за исполнение распоряжения во...,"[03 июня 2013 года, 1370196000.0]",NaN,NaN,NaN,NaN
2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"[24.12.20, 1608750000.0]",Провести рефакторинг программного обеспечения ...,Ответственным за исполнение настоящего приказа...,Отдел разработки систем,...,NaN,NaN,NaN,NaN,[Контроль над исполнением распоряжения оставит...,"[08 окт 2014 г., 1412704800.0]",NaN,NaN,NaN,NaN


In [ ]:
def get_date(x):
    processor = Processor([DATE])
    result = processor(x)
    if result.matches:
        result = f"{result.matches[0].referent.day}.{result.matches[0].referent.month}.{result.matches[0].referent.year}"
    else:
        for index, letter in enumerate(x):
          if letter.isdigit():
              result = x[index:]
              break
    return result

In [ ]:
df["Date_sig"] = raw_df["signatory_and_date"].apply(lambda x: get_date(x)) # добавляем дату подписания
first_column = df.pop("Date_sig")
df.insert(3, "Date_sig", first_column) # для удобства перемещаем в начало таблицы

In [ ]:
df.head(3)

,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,Task2.task_responsibles_people,...,Task8.task_text,Task8.task_responsibles_people,Task8.task_responsibles_groups,Task8.task_deadline,Global_supervisor,Global_deadline,Task9.task_text,Task9.task_responsibles_people,Task9.task_responsibles_groups,Task9.task_deadline
id,,,,,,,,,,,,,,,,,,,,,
0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"[19 апр 15, 1429383600.0]",NaN,NaN,...,NaN,NaN,NaN,NaN,[Ответственность за исполнение распоряжения во...,"[11 апреля 2013 года, 1365616800.0]",NaN,NaN,NaN,NaN
1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"[24 октября 12, 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,Контроль над выполнением приказа возложить на ...,...,NaN,NaN,NaN,NaN,[Ответственность за исполнение распоряжения во...,"[03 июня 2013 года, 1370196000.0]",NaN,NaN,NaN,NaN
2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"[24.12.20, 1608750000.0]",Провести рефакторинг программного обеспечения ...,Ответственным за исполнение настоящего приказа...,...,NaN,NaN,NaN,NaN,[Контроль над исполнением распоряжения оставит...,"[08 окт 2014 г., 1412704800.0]",NaN,NaN,NaN,NaN
3,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Оплатить командировочные расходы.\nОпределить ...,10.12.18,Приказ,Контроль над выполнением настоящего распоряжен...,Бухгалтерия,"[26 июля 2019 г., 1564081200.0]",Провести аудит персональных данных в информаци...,Обеспечение выполнения возложить на специалист...,...,NaN,NaN,NaN,NaN,[Контроль над выполнением настоящего распоряже...,"[10.12.18, 1544382000.0]",NaN,NaN,NaN,NaN
4,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Согласовать смету на выполнение работ компание...,19.03.2016,Приказ,Контроль над выполнением настоящего распоряжен...,Бухгалтерия,"[12 янв 2019 г., 1547233200.0]",Выделить средства на проведение обучения.\nУст...,Ответственным за исполнение настоящего приказа...,...,NaN,NaN,NaN,NaN,[Контроль над выполнением настоящего распоряже...,"[19.03.2016, 1458327600.0]",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Подготовить отчеты для налоговой службы.\nПрис...,22.8.2016,Приказ,Ответственным за исполнение настоящего приказа...,Бухгалтерия,"[14 сен 2017 г., 1505329200.0]",Перечислить необходимые средства для проведени...,За контроль над исполнением распоряжения назна...,...,NaN,NaN,NaN,NaN,[Ответственным за исполнение настоящего приказ...,"[22 августа 2016 года, 1471806000.0]",NaN,NaN,NaN,NaN
9996,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить необходимое количество лицензий офисн...,26 июля 12,Приказ,Контроль над исполнением распоряжения оставить...,Служба ИТ,"[03 октября 2020 года, 1601665200.0]",Закупить необходимое количество клавиатур и ма...,Ответственным за контролем над исполнением выш...,...,Подключить и настроить видеокамеру для работы ...,За контроль над исполнением распоряжения назна...,Служба ИТ,"[07 апр 15, 1428346800.0]",[Контроль над исполнением распоряжения оставит...,"[26 июля 12, 1343239200.0]",Провести централизованную смену паролей для до...,Контроль над выполнением настоящего распоряжен...,Служба ИТ,"[08.10.2019, 1570474800.0]"
9997,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Скорректировать проектно-сметную документацию ...,10.02.18,Приказ,Контроль над выполнением настоящего распоряжен...,Отдел проект

## Добавляем исполнителей по задачам

In [ ]:
def get_executor(x):
       doc = nlp(x)
       for entity in doc.ents:
          if entity.label_ == "PER":
            return entity.text

In [ ]:
for i in df.columns:
  if "responsibles_people" in i:   
       df[f"Executor_{i[:6]}"] = df[i].apply(lambda x: get_executor(x) if x and isinstance(x, str) else None)

#### Выделяем ФИО подписанта

In [ ]:
def get_signatory(x):
  processor = Processor([PERSON])
  result = processor(x)
  return f"{result.matches[0].referent.lastname} {result.matches[0].referent.firstname} {result.matches[0].referent.middlename}"

df["Name_signatory"] = raw_df["signatory_and_date"].apply(lambda x: get_signatory(x))

In [ ]:
df.head(3)

,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,Task2.task_responsibles_people,...,executor_Task1.,executor_Task2.,executor_Task3.,executor_Task4.,executor_Task5.,executor_Task6.,executor_Task7.,executor_Task8.,executor_Task9.,Name_signatory
id,,,,,,,,,,,,,,,,,,,,,
0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"[19 апр 15, 1429383600.0]",NaN,NaN,...,Кравцову И.Е.,None,None,None,None,None,None,None,None,СЕРДИТЫЙ А И
1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"[24 октября 12, 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,Контроль над выполнением приказа возложить на ...,...,Кима Юрия Ивановича,Сергеева Андрея,Кимом Юрием Ивановичем,None,None,None,None,None,None,СЕРДИТЫЙ А И
2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"[24.12.20, 1608750000.0]",Провести рефакторинг программного обеспечения ...,Ответственным за исполнение настоящего приказа...,...,Абаимовым Симоном,Абаимова Симона,Абашкина Ореста,Абалкина Зураба,Андреевым С.Е.,Андреева С.Е.,None,None,None,СОЗОНОВ С В


#### Выделяем должность подписанта

In [ ]:
def get_position_of_signatory(x):
      x = x.lower().replace(".", ' ')
      signatories = (i.lower() for i in df["Name_signatory"])
      for i in signatories:
        if i in x:
          position = x.replace(i, '')
          return position

In [ ]:
df["Position_of_signatory"] = df["Signatory"].apply(lambda x: get_position_of_signatory(x))

In [ ]:
df.head(3)

,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,Task2.task_responsibles_people,...,executor_Task2.,executor_Task3.,executor_Task4.,executor_Task5.,executor_Task6.,executor_Task7.,executor_Task8.,executor_Task9.,Name_signatory,Position_of_signatory
id,,,,,,,,,,,,,,,,,,,,,
0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"[19 апр 15, 1429383600.0]",NaN,NaN,...,None,None,None,None,None,None,None,None,СЕРДИТЫЙ А И,и о генерального директора
1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"[24 октября 12, 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,Контроль над выполнением приказа возложить на ...,...,Сергеева Андрея,Кимом Юрием Ивановичем,None,None,None,None,None,None,СЕРДИТЫЙ А И,и о генерального директора
2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"[24.12.20, 1608750000.0]",Провести рефакторинг программного обеспечения ...,Ответственным за исполнение настоящего приказа...,...,Абаимова Симона,Абашкина Ореста,Абалкина Зураба,Андреевым С.Е.,Андреева С.Е.,None,None,None,СОЗОНОВ С В,генеральный директор


In [ ]:
def get_supervisors_name(x):
       res = []
       x = ",".join(x)
       doc = nlp(x)
       for entity in doc.ents:
          if entity.label_ == "PER":
            res.append(entity.text)
       return res

In [ ]:
df["Names_global_supervisor"] = df["Global_supervisor"].apply(lambda x: get_supervisors_name(x) if x else None)

In [ ]:
all.to_csv("17.01.2023.csv") # промежуточно сохраняем

# Создаем новую сущность для нахождения должности в задачах

In [ ]:
# Создаем для новых NER (должность - POST) обучающую разметку

In [ ]:
 train = [("Ответственность за исполнение распоряжения возложить на специалиста первой категории административно-хозяйственного отдела Кравцову И.Е.", {'entities': [(56, 84, 'POST')]}),
  ("Ответственность за исполнение распоряжения возложить на специалиста первой категории административно-хозяйственного отдела Кравцову И.Е.", {'entities': [(56, 84, 'POST')]}),
  ("Ответственность за исполнение распоряжения возложить на специалиста третей категории бухгалтерии Александрова А.С.", {'entities': [(56, 84, 'POST')]}),
  ("За контроль над исполнением распоряжения назначить ответственным специалиста третей категории бухгалтерии Александрова А.С.", {'entities': [(65, 93, 'POST')]}),
  ("Ответственным за исполнение настоящего приказа назначить специалиста первой категории бухгалтерии Миронову Т.П.", {'entities': [(57, 84, 'POST')]}),
  ("За контроль и выполнение назначить ответственным специалиста второй категории бухгалтерии Смирнову О.А.", {'entities': [(49, 77, 'POST')]}),
  ("Ответственным за исполнение настоящего приказа назначить главного бухгалтера Семенову М.С.", {'entities': [(57, 76, 'POST')]}),
  ("Ответственным за контролем над исполнением вышеизложенных задач назначить технического писателя Абалкина Зураба", {'entities': [(74, 96, 'POST')]}),
  ("Контроль над исполнением распоряжения оставить за специалистом второй категории отдела разработки пользовательских интерфейсов Поповым А.А.", {'entities': [(50, 79, 'POST')]}),
  ("Ответственным за контролем над исполнением вышеизложенных задач назначить специалиста второй категории отдела архитектуры Симонова Н.С.", {'entities': [(74, 103, 'POST')]}),
  ("Контроль над выполнением приказа возложить на системного администратора Сергеева Андрея", {'entities': [(46, 71, 'POST')]}),
  ("Контроль над выполнением приказа возложить на программиста Абашкина Ореста",  {'entities': [(46, 58, 'POST')]}),
  ("За контроль и выполнение назначить ответственным главного бухгалтера Семенову М.С.",  {'entities': [(49, 68, 'POST')]}),
  ("Обеспечение выполнения возложить на мастера по ремонту компьютеров Абалакина Михаила",  {'entities': [(36, 66, 'POST')]}),
  ("Обеспечение выполнения возложить на руководителя Службы ИТ: Р.Е.Артюхина",  {'entities': [(36, 58, 'POST')]}),
  ("Обеспечение выполнения возложить на тимлид проектной команды Абаимова Симона",  {'entities': [(36, 60, 'POST')]}),
  ("Ответственным за контролем над исполнением вышеизложенных задач назначить специалиста первой категории Службы ИТ Александрова Александра Ивановича,",  {'entities': [(74, 112, 'POST')]}),
  ("Контроль над выполнением настоящего приказа возложить на руководителя отдела разработки систем Департамента разработки Петрова В.М.,",  {'entities': [(58, 95, 'POST')]}),
  ("Ответственность за исполнение распоряжения возложить на архитектора программного обеспечения Абатурова Бориса",  {'entities': [(56, 92, 'POST')]}),
  ("Ответственность за исполнение распоряжения возложить на руководителя Административно-хозяйственного отдела С.С.Галкина",  {'entities': [(56, 106, 'POST')]}), 
  
  ]

In [ ]:
str =  "Ответственность за исполнение распоряжения возложить на руководителя Административно-хозяйственного отдела С.С.Галкина"

In [ ]:
str[56:106]

'руководителя Административно-хозяйственного отдела'

In [ ]:
for i in train:
  print(i[0][i[1]['entities'][0][0]:i[1]['entities'][0][1]])

специалиста первой категории
специалиста первой категории
специалиста третей категории
специалиста третей категории
специалиста первой категори
специалиста второй категории
главного бухгалтера
технического писателя 
специалистом второй категории
специалиста второй категории 
системного администратора
программиста
главного бухгалтера
мастера по ремонту компьютеров
руководителя Службы ИТ
тимлид проектной команды
специалиста первой категории Службы ИТ
уководителя отдела разработки систем 
архитектора программного обеспечения
руководителя Административно-хозяйственного отдела


In [ ]:
nlp.remove_pipe('ner') # удаляем NER 

('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f4ada27ec80>)

Создаем собственную NER == POST

In [ ]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("en")
    # ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp 

In [ ]:
import random    
import datetime as dt
from spacy.training import Example
from spacy import displacy

nlp = create_blank_nlp(train)
optimizer = nlp.begin_training()  
for i in range(9):
    random.shuffle(train)
    losses = {}
    for text, annotations in train:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i} - {dt.datetime.now()}", losses)

/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Ответственным за исполнение настоящего приказа наз..." with entities "[(57, 84, 'POST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Контроль над выполнением настоящего приказа возлож..." with entities "[(58, 95, 'POST')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Ответственным за контролем над исполнением вышеизл..

Losses at iteration 0 - 2023-01-17 20:58:37.308495 {'ner': 95.35557748138672}
Losses at iteration 1 - 2023-01-17 20:58:37.877258 {'ner': 28.426277970033052}
Losses at iteration 2 - 2023-01-17 20:58:38.402684 {'ner': 31.96026306755713}
Losses at iteration 3 - 2023-01-17 20:58:38.919256 {'ner': 71.52810063198692}
Losses at iteration 4 - 2023-01-17 20:58:39.383004 {'ner': 18.042374138169706}
Losses at iteration 5 - 2023-01-17 20:58:39.872037 {'ner': 12.9574377124191}
Losses at iteration 6 - 2023-01-17 20:58:40.309377 {'ner': 12.447126264224348}
Losses at iteration 7 - 2023-01-17 20:58:40.746443 {'ner': 10.165720492749916}
Losses at iteration 8 - 2023-01-17 20:58:41.212818 {'ner': 7.37114058463739}


# Добавляем исполнителей по задачам на основании новой сущности "POST" в spacy

In [ ]:
def get_post(x):
  doc = nlp(x)
  for ent in doc.ents:
    return ent.text

In [ ]:
# import pymorphy2 # хотели получить в именительном падеже, но поняли, что потом будет сложно использовать
 
# def norm(x):
#     morph = pymorphy2.MorphAnalyzer()
#     p = morph.parse(x)[0]
#     return p.normal_form


In [ ]:
for i in df.columns:
  if "responsibles_people" in i:   
       df[f"responsibles_people_{i[:6]}"] = df[i].apply(lambda x: get_post(x) if x and isinstance(x, type(str)) else None)

In [ ]:
df["responsibles_people_Task3."].unique()

array([None, 'специалистом третьей категории', 'собой',
       'специалистом первой категории',
       'руководителя отдела проектирования',
       'специалиста первой категории', 'главного бухгалтера',
       'специалиста третьей категории', 'системного администратора',
       'специалиста второй категории', 'администратора баз',
       'архитектора программного обеспечения',
       'руководителя Административно-', 'руководителя Службы ИТ',
       'специалистом второй категории', 'мастера по ремонту компьютеров',
       'мастером по ремонту компьютеров', 'тимлид проектной команды',
       'руководителя отдела', 'специалистом третей категории',
       'администратора вычислительной сети',
       'специалиста третей категории', 'руководителя отдела тестирования',
       'руководителем Службы', 'технического писателя',
       'руководителя отдела разработки', 'тимлидами проектной команды',
       'системным администратором', 'руководителя Службы',
       'руководителя отдела эксплуатации

In [ ]:
#  df["responsibles_people_Task1."][df["responsibles_people_Task1."] == 'руководителя Административно-'].index.tolist()

In [ ]:
df.head(3)

,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,Task2.task_responsibles_people,...,responsibles_people_Task1.,responsibles_people_Task2.,responsibles_people_Task3.,responsibles_people_Task4.,responsibles_people_Task5.,responsibles_people_Task6.,responsibles_people_Task7.,responsibles_people_Task8.,responsibles_people_Task9.,responsibles_people_respon
id,,,,,,,,,,,,,,,,,,,,,
0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"[19 апр 15, 1429383600.0]",NaN,NaN,...,специалиста первой категории,None,None,None,None,None,None,None,None,None
1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"[24 октября 12, 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,Контроль над выполнением приказа возложить на ...,...,специалиста третьей категории,системного администратора,специалистом третьей категории,None,None,None,None,None,None,None
2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"[24.12.20, 1608750000.0]",Провести рефакторинг программного обеспечения ...,Ответственным за исполнение настоящего приказа...,...,тимлидами проектной команды,тимлид проектной команды,программиста,технического писателя,специалистом третьей категории,специалиста третьей категории,None,None,None,None


In [ ]:
df.to_csv("res_18.01.2023_23.24.csv")

In [ ]:
df["Global_supervisor"]

id
0       [Ответственность за исполнение распоряжения во...
1       [Ответственность за исполнение распоряжения во...
2       [Контроль над исполнением распоряжения оставит...
3       [Контроль над выполнением настоящего распоряже...
4       [Контроль над выполнением настоящего распоряже...
                              ...                        
9995    [Ответственным за исполнение настоящего приказ...
9996    [Контроль над исполнением распоряжения оставит...
9997    [Контроль над выполнением настоящего распоряже...
9998    [Обеспечение выполнения возложить на специалис...
9999    [Ответственность за исполнение распоряжения во...
Name: Global_supervisor, Length: 10000, dtype: object

In [ ]:
def get_posts(x):
  res = []
  x = ",".join(x)
  doc = nlp(x)
  for ent in doc.ents:
    res.append(ent.text)
  return res

In [ ]:
def get_supervisors_name(x):
       res = []
       x = ",".join(x)
       doc = nlp(x)
       for entity in doc.ents:
          if entity.label_ == "PER":
            res.append(entity.text)
       return res

In [ ]:
df["Posts_global_supervisors"] = df["Global_supervisor"].apply(lambda x: get_posts(x) if x else None)

In [ ]:
df

,Signatory,Organization,Task1.task_text,Date_sig,Title,Task1.task_responsibles_people,Task1.task_responsibles_groups,Task1.task_deadline,Task2.task_text,Task2.task_responsibles_people,...,responsibles_people_Task4.,responsibles_people_Task5.,responsibles_people_Task6.,responsibles_people_Task7.,responsibles_people_Task8.,responsibles_people_Task9.,responsibles_people_respon,Nasmes_global_supervisor,Names_global_supervisor,Posts_global_supervisors
id,,,,,,,,,,,,,,,,,,,,,
0,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить книги. Срок завершения: 19 апр 15.,11.4.2013,Приказ,Ответственность за исполнение распоряжения воз...,Административно-хозяйственный отдел,"[19 апр 15, 1429383600.0]",NaN,NaN,...,None,None,None,None,None,None,None,None,[Кравцову И.Е.],[специалиста первой категории]
1,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Провести анализ поставщиков для замены вышедше...,3.6.2013,Приказ,Ответственность за исполнение распоряжения воз...,Служба ИТ,"[24 октября 12, 1351015200.0]",Выполнить ремонт кабеля сети передачи данных м...,Контроль над выполнением приказа возложить на ...,...,None,None,None,None,None,None,None,None,"[Кима Юрия Ивановича, Сергеева Андрея, Контрол...","[специалиста третьей категории, системного адм..."
2,генеральный директор Созонов С.В.,"ООО ""Мегасофт""",Провести рефакторинг программного обеспечения ...,8.10.2014,Приказ,Контроль над исполнением распоряжения оставить...,Отдел разработки систем,"[24.12.20, 1608750000.0]",Провести рефакторинг программного обеспечения ...,Ответственным за исполнение настоящего приказа...,...,технического писателя,специалистом третьей категории,специалиста третьей категории,None,None,None,None,None,"[Абаимовым Симоном, Ответственным, Абаимова Си...","[тимлидами проектной команды, тимлид проектной..."
3,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Оплатить командировочные расходы.\nОпределить ...,10.12.18,Приказ,Контроль над выполнением настоящего распоряжен...,Бухгалтерия,"[26 июля 2019 г., 1564081200.0]",Провести аудит персональных данных в информаци...,Обеспечение выполнения возложить на специалист...,...,специалиста второй категории,специалиста второй категории,None,None,None,None,None,None,"[Смирнову, Смирнову О.А.]","[специалиста третей категории, специалиста вто..."
4,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Согласовать смету на выполнение работ компание...,19.03.2016,Приказ,Контроль над выполнением настоящего распоряжен...,Бухгалтерия,"[12 янв 2019 г., 1547233200.0]",Выделить средства на проведение обучения.\nУст...,Ответственным за исполнение настоящего приказа...,...,None,None,None,None,None,None,None,None,"[Ответственным, Смирнову О.А.]",[специалиста второй категории]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Подготовить отчеты для налоговой службы.\nПрис...,22.8.2016,Приказ,Ответственным за исполнение настоящего приказа...,Бухгалтерия,"[14 сен 2017 г., 1505329200.0]",Перечислить необходимые средства для проведени...,За контроль над исполнением распоряжения назна...,...,None,None,None,None,None,None,None,NaN,"[Семенову, Александрова А.С.]","[главного бухгалтера, специалиста третей катег..."
9996,и.о. генерального директора Сердитый А.И.,"ООО ""Мегасофт""",Закупить необходимое количество лицензий офисн...,26 июля 12,Приказ,Контроль над исполнением распоряжения оставить...,Служба ИТ,"[03 октября 2020 года, 1601665200.0]",Закупить необходимое количество клавиатур и ма...,Ответственным за контролем над исполнением выш...,...,специалиста второй категории,None,None,мастера по ремонту компьютеров,специалиста третьей категории,None,None,NaN,"[Кимом Юрием Ивановичем, Ответственным, Сергее...","[специалистом третьей категории, системного ад..."
9997,и.о. генерального директора Кравцов С.С.,"ООО ""Мегасофт""",Скорректировать проектно-сметную документацию ...,10.02.18,Приказ,Контроль над выполнение

In [ ]:
df.to_csv("markup_18.01.2023.csv")

In [ ]:
df.columns

Index(['id', 'Task1.task_text', 'Task1.task_responsibles_people',
       'Task1.task_responsibles_groups', 'Task1.task_deadline',
       'Task2.task_text', 'Task2.task_responsibles_people',
       'Task2.task_responsibles_groups', 'Task2.task_deadline',
       'Task3.task_text', 'Task3.task_responsibles_people',
       'Task3.task_responsibles_groups', 'Task3.task_deadline',
       'Task4.task_text', 'Task4.task_responsibles_people',
       'Task4.task_responsibles_groups', 'Task4.task_deadline',
       'Task5.task_text', 'Task5.task_responsibles_people',
       'Task5.task_responsibles_groups', 'Task5.task_deadline',
       'Task6.task_text', 'Task6.task_responsibles_people',
       'Task6.task_responsibles_groups', 'Task6.task_deadline',
       'Task7.task_text', 'Task7.task_responsibles_people',
       'Task7.task_responsibles_groups', 'Task7.task_deadline',
       'Task8.task_text', 'Task8.task_responsibles_people',
       'Task8.task_responsibles_groups', 'Task8.task_deadline',
  

У нас есть:<br> 
id,<br> 'Task1.task_text' - текст задачи,<br> 'Task1.task_responsibles_people' - ответствнный за задачу,<br> 'Task1.task_responsibles_groups' - ответственный отдел за задачу,<br> 'Task1.task_deadline' - срок выполнения задачи, (по каждой задаче),<br> 'ORG' -  название организации,<br>
'title' - название документа,<br> 'signatory' - подписант<br> 'date_sig' - дата подписания,<br> 'position_of_signatory' - должность подписанта,<br> 'Global_supervisor' - главный ответственный,<br> 'Global_deadline' - общий срок выполнения

In [ ]:
df["Global_supervisor"]

0       ['Ответственность за исполнение распоряжения в...
1       ['Ответственность за исполнение распоряжения в...
2       ['Контроль над исполнением распоряжения остави...
3       ['Контроль над выполнением настоящего распоряж...
4       ['Контроль над выполнением настоящего распоряж...
                              ...                        
9995    ['Ответственным за исполнение настоящего прика...
9996    ['Контроль над исполнением распоряжения остави...
9997    ['Контроль над выполнением настоящего распоряж...
9998    ['Обеспечение выполнения возложить на специали...
9999    ['Ответственность за исполнение распоряжения в...
Name: Global_supervisor, Length: 10000, dtype: object